First we create the connection to our mysql server. For that, let's install mysql-connector for this matter.

In [0]:
!pip install mysql-connector

Once done, we can proceed importing the library for our use.

In [0]:
import mysql.connector
from mysql.connector import Error

Next step is to create a function we will use to create the connection to our server.

In [0]:
def connect():
    try:
        conn = mysql.connector.connect(host='your_host',
                                       port='your_port',
                                       database='your_db',
                                       user='your_user',
                                       password='your_password' )
        if conn.is_connected():
            print('Connected to MySQL database')
            return conn

    except Error as e:
        print(e)


Now that our connection is set and ready to be used, we will proceed calling our function and running our query to select the ips we want the latitude and longitud for.

In [0]:
connection = connect()

cursor = connection.cursor(buffered=True)

sql_select_ip = """
                    your_query_here
                """

cursor.execute(sql_select_ip)
assert isinstance(cursor.fetchall, object)

You may ask yourself what do we do now with  the data queried above. Why don't we just put it into a good old pandas' dataframe?

In [0]:
!pip install pandas

In [0]:
import pandas as pd

df = pd.read_sql_query(sql_select_ip, connection)

To continue we will create a field in the dataframe which we will use for our call the website where we will get the results from.
This website will be http://ip-api.com. This website allows us to choose between a nice amount of data related to our ip. To choose we can go to https://ip-api.com/docs/api:json and check the data needed. They will then provide us with a url that we can set up to create our calls.

We will use the following:

```
'http://ip-api.com/json/' + ip + '?fields=status,city,lat,lon'
```




In [0]:
df['ip_api'] = ['http://ip-api.com/json/' + i for i in df['ipclient'] + '?fields=status,city,lat,lon']

This website will provide a JSON with the answer to our request. With this JSON we will populate a list previously set as empty to make sure no data is inside.

Also, let's set a counter to 0 for the next step.

In [0]:
json_ip = []

count_i = 0 

With all set we just need to create a loop where the calls to the website will be made from. For it we need a couple of libraries: tqdm and requests.

**tqdm** is not essential to the result, but gives us a way to know the progress of the loop so we make sure it is actually running.

**requests** is indeed the essential part of the loop. It will call the website via api and retrieve the JSON required by us.

There is an inconvenient with ip_api.com. For the free use of it, they only allow us to make 45 calls per minute, it is a bit crappy but understandable, in this capitalized world, most of us want (need) to monetize in some way our products.

Now we now what do we use the counter for. Each call, the counter will sum up 1, and if we reach 40 we will wait 60 seconds. Easy.

In [0]:
!pip install tqdm
!pip install requests
!pip install time

In [0]:
import tqdm
import requests
import time

for url in tqdm(df.ip_api, leave=False):
    request = requests.get(url)
    json_ip.append(request.content)
    count_i += 1
    if count_i == 40:
        time.sleep(60)
        count_i = 0

Now that we have everything we need, we can store the resulting data in a csv in case we have some issue and we want to study it.

In [0]:
!pip install csv

In [0]:
import csv

with open('json_ip.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(json_ip)

with open('json_ip.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for i in reader:
        json_ip = i
        break

Let's now put all our data together creating a column in our dataframe to fill with the json's resulting of above.

In [0]:
df['json_ip'] = json_ip

We are almost there. 

We now do some good use of pandas and python's good list comprehension. What we want now is to extract the data needed from the JSON file, and with some splitting we can get a great result. We need to make sure we store the results in our dataframe. For this case, we just want the latitude, longitud and the city of the ips we have.

In [0]:
df['city'] = [result_ip.split('"city":"')[1].split('","lat"')[0] for result_ip in df['json_ip']]
df['lat'] = [float(result_ip.split(',"lat":')[1].split(',"lon":')[0]) for result_ip in df['json_ip']]
df['lon'] = [float(result_ip.split(',"lon":')[1].split("}'")[0]) for result_ip in df['json_ip']]

All we are missing right now is how and where we storage our resulting data frame.

How: we don't need all the columns created along the exercise, so we do some slicing having a more compact and readable dataframe.

Where: with a simple csv in a folder so we can manipulate if needed.

In [0]:
df = df[['ipclient', 'city', 'lat', 'lon']]

df.to_csv('your_file.csv', header=True, sep=';', encoding='utf-8') #make sure you end the files name with .csv